In [ ]:
dataset_path = '/content/A2_Data.csv'

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import random
import requests
from io import BytesIO
import pandas as pd
import pandas as pd
import nltk
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO


In [ ]:


dataset_path = '/content/A2_Data.csv'
data = pd.read_csv(dataset_path)

image_paths = data['Image'].str.strip("[]").str.split(", ")

image_paths = [item.strip("'") for sublist in image_paths for item in sublist]

print(image_paths)


['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71HSx4Y-5dL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71dVsYejzTL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71domStNfIL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71Md5ihUFLL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71Isri9SEaL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71w8aOdrTuL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81dxkALs4CL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81VixyruzDL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/51IWGFH3IPL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81p1k-2194L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71cS64LddWL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71z9biVe+ML._SY88.jpg', 'https://images-na.ssl-images-amazon.

Question 1

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import random
import requests
from io import BytesIO
import pandas as pd

def preprocess_image(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = response.content
        image_array = np.frombuffer(image_data, np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224))
        if random.choice([True, False]):
            image = cv2.flip(image, 1)
        brightness_factor = np.random.uniform(0.7, 1.3)
        image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
        contrast_factor = np.random.uniform(0.7, 1.3)
        enhancer = ImageEnhance.Contrast(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
        image = np.array(enhancer.enhance(contrast_factor))
        angle = np.random.uniform(-15, 15)
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
        image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
        image = cv2.GaussianBlur(image, (5, 5), 0)

        return image
    else:
        print("Failed to fetch image from URL:", image_url)
        return None

dataset_path = '/content/A2_Data.csv'
data = pd.read_csv(dataset_path, sep='\t', names=['signal_number', 'Image', 'Review_text'])

for index, row in data.iterrows():
    image_paths = str(row['Image']).strip("[]").split(", ")
    image_paths = [path.strip("'") for path in image_paths if path != 'nan']
    for image_path in image_paths:
        if image_path.startswith("http://") or image_path.startswith("https://"):
            preprocessed_image = preprocess_image(image_path)
            if preprocessed_image is not None:
                print("Preprocessed image shape:", preprocessed_image.shape)
        else:
            print("Invalid URL:", image_path)
            data.drop(index, inplace=True)
            break

data.to_csv('/content/Modified_A2_Data.csv', sep='\t', index=False)

print(data)


                                          signal_number  Image  Review_text
0                                    ,Image,Review Text    NaN          NaN
1     3452,['https://images-na.ssl-images-amazon.com...    NaN          NaN
2     1205,"['https://images-na.ssl-images-amazon.co...    NaN          NaN
3     1708,['https://images-na.ssl-images-amazon.com...    NaN          NaN
4     The greatest benefit has been when writing mus...    NaN          NaN
...                                                 ...    ...          ...
1659  1547,"['https://images-na.ssl-images-amazon.co...    NaN          NaN
1660  1004,['https://images-na.ssl-images-amazon.com...    NaN          NaN
1661  I hope that I will never have to use the warra...    NaN          NaN
1662  This is a pity because this microphone is just...    NaN          NaN
1663  1306,['https://images-na.ssl-images-amazon.com...    NaN          NaN

[1664 rows x 3 columns]


In [ ]:


data = pd.read_csv(dataset_path, header=None, names=['Image', 'Review Text'])

image_paths = data['Image'].str.strip("[]").str.split(", ")
image_paths = [item.strip("'") for sublist in image_paths for item in sublist]

resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        img = img.resize((224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        features = resnet_model.predict(x)
        return features.flatten()
    except Exception as e:
        print(f"Error processing image at {image_url}: {e}")
        return None

image_features = []
for image_path in image_paths:
    features = extract_features(image_path)
    if features is not None:
        image_features.append(features)

image_features = np.array(image_features)

print("Shape of extracted features:", image_features.shape)



94765736/94765736 [==============================] - 0s 0us/step
Error processing image at Image: Invalid URL 'Image': No scheme supplied. Perhaps you meant https://Image?
1/1 [==============================] - 0s 222ms/step
Error processing image at https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg: 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg
Error processing image at https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg: 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg
1/1 [==============================] - 0s 173ms/step
Error processing image at https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg: 404 Client Error: Not Found for url: https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg
Error processing image at https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg: 404 Cl

Normalizing extracted features

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
image_features_normalized = scaler.fit_transform(image_features)

print("Shape of normalized features:", image_features_normalized.shape)


Shape of normalized features: (1640, 2048)


In [ ]:
import pickle


with open('normalized_image_features.pkl', 'wb') as f:
    pickle.dump(image_features_normalized, f)

with open('image_paths.pkl', 'wb') as f:
    pickle.dump(image_paths, f)


In [ ]:
import pickle

with open('image_features.pkl', 'wb') as f:
    pickle.dump(image_features, f)


Normalizing Review Text

In [ ]:


data = pd.read_csv(dataset_path, header=None, skiprows=1, names=['Image', 'Review Text'])

data['Review Text'] = data['Review Text'].fillna('')

text_reviews = data['Review Text']

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    text = str(text).lower()


    tokens = word_tokenize(text)


    tokens = [token for token in tokens if token not in string.punctuation and token not in stop_words]


    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]


    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(token) for token in tokens]


    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

preprocessed_reviews = [preprocess_text(review) for review in text_reviews]

for i, review in enumerate(preprocessed_reviews):
    print(f"Review {i+1}: {review}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Review 1: loving vintage spring vintage strat good tension great stability floating bridge want spring way go
Review 2: work great guitar bench mat rugged enough abuse take care take care make organization workspace much easier screw wo n't roll around color good
Review 3: use everything acoustic bass ukulele know smaller model available uke violin etc n't yet ordered work smaller instrument one n't extend foot maximum width 're gentle instrument grippy material keep secure greatest benefit writing music computer needing set guitar use keyboard/mouse easier hanging stand several gave one friend christmas well 've used mine stage fold small enough fit right gig bag
Review 4: great price good quality n't quite match radius sound hole close enough
Review 5: bought bass split time primary bass dean edge might winning bass boost outstanding active pickup really allow adjust sound want recommend anyone 're beginner like long ago 's excellent bass start 're tour and/or music making money bass

TF-IDF

In [ ]:
import pandas as pd
import numpy as np
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

dataset_path = '/content/A2_Data.csv'
data = pd.read_csv(dataset_path, header=None, skiprows=1, names=['Image', 'Review Text'])

data['Review Text'] = data['Review Text'].fillna('')

text_reviews = data['Review Text']

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
tokenized_reviews = [word_tokenize(review.lower()) for review in text_reviews]
filtered_reviews = [[word for word in review if word.isalnum() and word not in stop_words] for review in tokenized_reviews]

with open('filtered_reviews.pkl', 'wb') as f:
    pickle.dump(filtered_reviews, f)

def calculate_tfidf(review, filtered_reviews, num_documents):
    term_frequencies = {}
    for word in review:
        term_frequencies[word] = term_frequencies.get(word, 0) + 1

    review_tfidf = {}
    for word, freq in term_frequencies.items():
        tf = freq / len(review)
        df = sum(1 for doc in filtered_reviews if word in doc)
        idf = np.log(num_documents / (1 + df))
        review_tfidf[word] = tf * idf

    return review_tfidf

num_documents = len(filtered_reviews)
tfidf_scores = [calculate_tfidf(review, filtered_reviews, num_documents) for review in filtered_reviews]

with open('tfidf_scores.pkl', 'wb') as f:
    pickle.dump(tfidf_scores, f)

print("TF-IDF scores for the first review:")
for word, score in tfidf_scores[0].items():
    print(f"{word}: {score}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TF-IDF scores for the first review:
loving: 0.35322115776986907
vintage: 0.5284421733087595
springs: 0.6821327746338777
strat: 0.21978915775586083
good: 0.10532527401283706
tension: 0.31403538010972787
great: 0.07575427705680808
stability: 0.34106638731693883
floating: 0.36809739452414975
bridge: 0.22542631695773171
want: 0.1754059439977388
way: 0.1754059439977388
go: 0.1651292320092549


FINAL QUESTION

In [ ]:


def preprocess_image(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img = img.resize((224, 224))
            img_array = np.array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            return img_array
        else:
            print("Failed to fetch image from URL:", image_url)
            return None
    except Exception as e:
        print(f"Error processing image at {image_url}: {e}")
        return None

def preprocess_text(text):
    text = str(text).lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in string.punctuation and token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

image_url = input("Enter the image URL: ")
review_text = input("Enter the review text: ")

input_image_features = preprocess_image(image_url)

input_text_tokens = preprocess_text(review_text)

with open('normalized_image_features.pkl', 'rb') as f:
    image_features_normalized = pickle.load(f)

with open('image_paths.pkl', 'rb') as f:
    image_paths = pickle.load(f)

with open('filtered_reviews.pkl', 'rb') as f:
    filtered_reviews = pickle.load(f)

with open('tfidf_scores.pkl', 'rb') as f:
    tfidf_scores = pickle.load(f)

def image_retrieval(input_features, image_features, image_paths, top_k=3):
    similarities = cosine_similarity(input_features, image_features)
    top_indices = similarities.argsort()[0][-top_k:][::-1]
    return [(image_paths[i], similarities[0][i]) for i in top_indices]

def text_retrieval(input_tfidf, tfidf_scores, top_k=3):
    similarities = [cosine_similarity([list(input_tfidf.values())], [list(score.values())])[0][0] for score in tfidf_scores]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(filtered_reviews[i], similarities[i]) for i in top_indices]

print("USING IMAGE RETRIEVAL")
top_image_results = image_retrieval(input_image_features, image_features_normalized, image_paths)
for i, (image_url, similarity) in enumerate(top_image_results, start=1):
    print(f"{i}) Image URL: {image_url}\n   Cosine similarity: {similarity}")

print("\nUSING TEXT RETRIEVAL")
input_tfidf = calculate_tfidf(input_text_tokens, filtered_reviews)
top_text_results = text_retrieval(input_tfidf, tfidf_scores)
for i, (review, similarity) in enumerate(top_text_results, start=1):
    print(f"{i}) Review: {' '.join(review)}\n   Cosine similarity: {similarity}")

composite_similarity_scores = [(image_similarity + text_similarity) / 2 for (_, image_similarity), (_, text_similarity) in zip(top_image_results, top_text_results)]

ranked_pairs = sorted(zip(top_image_results, top_text_results, composite_similarity_scores), key=lambda x: x[2], reverse=True)

print("\nTop-ranked pairs along with composite similarity scores:")
for i, ((image_url, image_similarity), (review, text_similarity), composite_similarity) in enumerate(ranked_pairs, start=1):
    print(f"{i}) Image URL: {image_url}\n   Review: {' '.join(review)}\n   Composite similarity score: {composite_similarity}")



Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
Enter the review text: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
USING IMAGE RETRIEVAL


ValueError: shapes (1,224,224,3) and (1640,2048) not aligned: 3 (dim 3) != 1640 (dim 0)